# Eksempler på bruk av Klass fra Python 
Hvordan hente:
1. klassifikasjon
2. versjon
3. endringer
4. korrespondansetabell som CSV
5. variant som CSV
6. korrespondansetabell som JSON med biblioteket jmespath
7. variant som som JSON med biblioteket jmespath

Klass - Klassifikasjoner og kodelister. 
Se også Klass API dokumentasjon: [Klass API guide](https://data.ssb.no/api/klass/v1/api-guide.html)

### 1. Et enkelt eksempel: Klassifikasjon - dagens kommuner

Vi importerer Python Pandas, som kan angis som pd

In [1]:
# Pandas gjør dataanalyse i Python enklere
import pandas as pd

# requests for å hente data
import requests

Eksempelet henter gyldige koder fra Standard for kommuneinndeling.

Adressen i Klass er https://www.ssb.no/klass/klassifikasjoner/131/

131 = ID til klassifikasjonen

Vi henter gyldige koder som JSON via Klass API med [codesAt](https://data.ssb.no/api/klass/v1/api-guide.html#_codesat):

In [2]:
URL1 = 'https://data.ssb.no/api/klass/v1/classifications/131/codesAt.json?date=2021-01-19'

Siden dette er json kan vi bruke [Pandas read_json()]('https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html'). Den kan ta nettadresser direkte.

In [3]:
dataframe1 = pd.read_json(URL1)
dataframe1

,codes
0,"{'code': '0301', 'parentCode': None, 'level': ..."
1,"{'code': '1101', 'parentCode': None, 'level': ..."
2,"{'code': '1103', 'parentCode': None, 'level': ..."
3,"{'code': '1106', 'parentCode': None, 'level': ..."
4,"{'code': '1108', 'parentCode': None, 'level': ..."
...,...
352,"{'code': '5441', 'parentCode': None, 'level': ..."
353,"{'code': '5442', 'parentCode': None, 'level': ..."
354,"{'code': '5443', 'parentCode': None, 'level': ..."
355,"{'code': '5444', 'parentCode': None, 'level': ..."


Alt ligger i en kolonne 'codes'. Vi må i tillegg bruke Pandas [json_normalize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html). 

Vi gir dataframe1\['codes'\] som input

In [4]:
kommuner2020 = pd.json_normalize(dataframe1['codes'])
kommuner2020

,code,parentCode,level,name,shortName,presentationName,validFrom,validTo,notes
0,0301,None,1,Oslo,,,None,None,
1,1101,None,1,Eigersund,,,None,None,
2,1103,None,1,Stavanger,,,None,None,
3,1106,None,1,Haugesund,,,None,None,
4,1108,None,1,Sandnes,,,None,None,
...,...,...,...,...,...,...,...,...,...
352,5441,None,1,Deatnu - Tana,,,None,None,
353,5442,None,1,Unjárga - Nesseby,,,None,None,
354,5443,None,1,Båtsfjord,,,None,None,
355,5444,None,1,Sør-Varanger,,,None,None,


### 2. Henter Standard for kommuneinndeling med endringer fra og med  1.1.2017 
 
Url til Klass API. Nå bruker vi [codes](https://data.ssb.no/api/klass/v1/api-guide.html#_codes) med fra-og-med dato. Id er fortsatt 131 og vi angir .json slik: 

In [5]:
URL2 = 'https://data.ssb.no/api/klass/v1/classifications/131/codes.json?from=2017-01-01'

to= parameteren for tid er ikke nødvendig og kan med fordel utelukkes om en vil ha med endringer. 

Leser URL2 som json til dataframe2

In [6]:
dataframe2 = pd.read_json(URL2)
dataframe2.head()

,codes
0,"{'code': '0101', 'parentCode': None, 'level': ..."
1,"{'code': '0104', 'parentCode': None, 'level': ..."
2,"{'code': '0105', 'parentCode': None, 'level': ..."
3,"{'code': '0106', 'parentCode': None, 'level': ..."
4,"{'code': '0111', 'parentCode': None, 'level': ..."


Vi bruker Pandas json_normalize

In [7]:
kommuner_fra2017 = pd.json_normalize(dataframe2['codes'])

In [8]:
kommuner_fra2017

,code,parentCode,level,name,shortName,presentationName,validFrom,validTo,validFromInRequestedRange,validToInRequestedRange,notes
0,0101,None,1,Halden,,,None,None,2017-01-01,2020-01-01,
1,0104,None,1,Moss,,,None,None,2017-01-01,2020-01-01,
2,0105,None,1,Sarpsborg,,,None,None,2017-01-01,2020-01-01,
3,0106,None,1,Fredrikstad,,,None,None,2017-01-01,2020-01-01,
4,0111,None,1,Hvaler,,,None,None,2017-01-01,2020-01-01,
...,...,...,...,...,...,...,...,...,...,...,...
731,5441,None,1,Deatnu - Tana,,,None,None,2020-01-01,None,
732,5442,None,1,Unjárga - Nesseby,,,None,None,2020-01-01,None,
733,5443,None,1,Båtsfjord,,,None,None,2020-01-01,None,
734,5444,None,1,Sør-Varanger,,,None,None,2020-01-01,None,


Vi bruker [drop]('https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html') for å fjerne unødvendige kolonner.

In [9]:
kommuner_fra2017 = kommuner_fra2017.drop(columns=['parentCode', 'level', 'shortName', 'presentationName'])
kommuner_fra2017.head()

,code,name,validFrom,validTo,validFromInRequestedRange,validToInRequestedRange,notes
0,0101,Halden,None,None,2017-01-01,2020-01-01,
1,0104,Moss,None,None,2017-01-01,2020-01-01,
2,0105,Sarpsborg,None,None,2017-01-01,2020-01-01,
3,0106,Fredrikstad,None,None,2017-01-01,2020-01-01,
4,0111,Hvaler,None,None,2017-01-01,2020-01-01,


Vi setter code som inneholder kommunenummer som index

In [10]:
kommuner_fra2017 = kommuner_fra2017.set_index('code')

In [11]:
kommuner_fra2017.tail()

,name,validFrom,validTo,validFromInRequestedRange,validToInRequestedRange,notes
code,,,,,,
5441,Deatnu - Tana,None,None,2020-01-01,None,
5442,Unjárga - Nesseby,None,None,2020-01-01,None,
5443,Båtsfjord,None,None,2020-01-01,None,
5444,Sør-Varanger,None,None,2020-01-01,None,
9999,Uoppgitt,None,None,2017-01-01,None,


Finn kommunenummer og gyldighetsperiode for Holmestrand. Se gyldighetsperiodene

In [12]:
kommuner_fra2017.query('name == "Holmestrand"')

,name,validFrom,validTo,validFromInRequestedRange,validToInRequestedRange,notes
code,,,,,,
0702,Holmestrand,None,None,2017-01-01,2018-01-01,
0715,Holmestrand,None,None,2018-01-01,2020-01-01,
3802,Holmestrand,None,None,2020-01-01,None,


## Eksempel 3 Endringer i klassifikasjon og framtidige endringer
Her bruker vi *changes* og paramteren *includeFuture=True* for å få med revering av regionreformen

In [25]:
URL3 = 'https://data.ssb.no/api/klass/v1/classifications/131/changes.json?from=2019-01-01&includeFuture=True'

In [26]:
dataframe3= pd.read_json(URL3)
dataframe3

,codeChanges
0,"{'oldCode': '1103', 'oldName': 'Stavanger', 'o..."
1,"{'oldCode': '1141', 'oldName': 'Finnøy', 'oldS..."
2,"{'oldCode': '1142', 'oldName': 'Rennesøy', 'ol..."
3,"{'oldCode': '1102', 'oldName': 'Sandnes', 'old..."
4,"{'oldCode': '1129', 'oldName': 'Forsand', 'old..."
...,...
429,"{'oldCode': '5441', 'oldName': 'Deatnu - Tana'..."
430,"{'oldCode': '5440', 'oldName': 'Berlevåg', 'ol..."
431,"{'oldCode': '5443', 'oldName': 'Båtsfjord', 'o..."
432,"{'oldCode': '5404', 'oldName': 'Vardø', 'oldSh..."


In [27]:
kommuneendringer_fra2019 = pd.json_normalize(dataframe3['codeChanges'])
kommuneendringer_fra2019

,oldCode,oldName,oldShortName,newCode,newName,newShortName,changeOccurred
0,1103,Stavanger,,1103,Stavanger,,2020-01-01
1,1141,Finnøy,,1103,Stavanger,,2020-01-01
2,1142,Rennesøy,,1103,Stavanger,,2020-01-01
3,1102,Sandnes,,1108,Sandnes,,2020-01-01
4,1129,Forsand,,1108,Sandnes,,2020-01-01
...,...,...,...,...,...,...,...
429,5441,Deatnu - Tana,,5628,Deatnu - Tana,,2024-01-01
430,5440,Berlevåg,,5630,Berlevåg,,2024-01-01
431,5443,Båtsfjord,,5632,Båtsfjord,,2024-01-01
432,5404,Vardø,,5634,Vardø,,2024-01-01


### Eksempel 4. Korrespondansetabell med ID 639
Standard for reiselivsregioner. ID hentes herfra https://www.ssb.no/klass/klassifikasjoner/527/korrespondanser/639 

 Det *enkleste* er å hente dette som CSV i stedet med [Pandas read_csv]('https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html')
 
 JSON-output her er litt mer kompleks, så det er ikke mulig å gjøre det slik som over.

In [28]:
URL4 = 'https://data.ssb.no/api/klass/v1/correspondencetables/639.csv'

CSV i Klass er ikke UTF-8. Vi må sette tegnsett til ISO-8859-1 og skilletegn som er semikolon. Kodene er av typen 'string'. Da får vi med 0 først i koden.

In [17]:
dataframe4 = pd.read_csv(URL4, encoding = "ISO-8859-1", sep=';', dtype='str')
dataframe4.head()

,sourceCode,sourceName,targetCode,targetName
0,03101,Oslo,0301,Oslo
1,11101,Stavangerregion,1101,Eigersund
2,11101,Stavangerregion,1103,Stavanger
3,11101,Stavangerregion,1108,Sandnes
4,11101,Stavangerregion,1111,Sokndal


### Eksempel 5. Variant med ID 1121
Eurostats gruppering av varetyper. ID hentes herfra https://www.ssb.no/klass/klassifikasjoner/6/varianter/1121. 

In [18]:
URL5 = 'https://data.ssb.no/api/klass/v1/variants/1121.csv'

Her detetekteres code automatisk som string. Dessuten er det nyttig at level er tall

In [19]:
dataframe5 = pd.read_csv(URL5, encoding = "ISO-8859-1", sep=';')
dataframe5.head(10)

,code,parentCode,level,name,shortName,notes,validFrom,validTo
0,05,E6,2,Bryting av steinkull og brunkull,NaN,NaN,NaN,NaN
1,06,E6,2,Utvinning av råolje og naturgass,NaN,NaN,NaN,NaN
2,07,E1,2,Bryting av metallholdig malm,NaN,NaN,NaN,NaN
3,08,E1,2,Bryting og bergverksdrift ellers,NaN,NaN,NaN,NaN
4,09,E1,2,Tjenester tilknyttet bergverksdrift og utvinning,NaN,NaN,NaN,NaN
5,10.1,E4,3,"Produksjon, bearbeiding og konservering av kjø...",NaN,NaN,NaN,NaN
6,10.2,E4,3,"Bearbeiding og konservering av fisk, skalldyr ...",NaN,NaN,NaN,NaN
7,10.3,E4,3,Bearbeiding og konservering av frukt og grønns...,NaN,NaN,NaN,NaN
8,10.4,E4,3,Produksjon av vegetabilske og animalske oljer ...,NaN,NaN,NaN,NaN
9,10.5,E4,3,Produksjon av meierivarer og iskrem,NaN,NaN,NaN,NaN


Her fjerner jeg helt tomme kolonner med [Pandas dropna]('https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html'), som legges i ny dataframe3_min

In [20]:
dataframe5_min = dataframe5.dropna(axis='columns', how='all')
dataframe5_min

,code,parentCode,level,name
0,05,E6,2,Bryting av steinkull og brunkull
1,06,E6,2,Utvinning av råolje og naturgass
2,07,E1,2,Bryting av metallholdig malm
3,08,E1,2,Bryting og bergverksdrift ellers
4,09,E1,2,Tjenester tilknyttet bergverksdrift og utvinning
...,...,...,...,...
76,E2,NaN,1,Investeringsvarer
77,E3,E5,2,Varige konsumvarer
78,E4,E5,2,Ikke-varige konsumvarer
79,E5,NaN,1,Konsumvarer


## Eksempel 6. Hente korrespondansetabell som json

Her bruker jeg jmespath, som er et 'JSON query language'. Se https://jmespath.org. For Python kan det installeres med 'pip install jmespath'

In [21]:
import jmespath

ModuleNotFoundError: No module named 'jmespath'

In [29]:
URL6 = 'https://data.ssb.no/api/klass/v1/correspondencetables/639.json'

In [30]:
payload = requests.get(url = URL6)

I URL-en har vi angitt at Klass skal gi json, men vi må også angi at innholdet skal være JSON. 

In [31]:
dataset6 = payload.json()

In [32]:
type(dataset6)

dict

In [33]:
dataset6

{'name': 'Reiselivsregioner 2020 - Kommuneinndeling 2020',
 'contactPerson': {'name': 'Aasestad, Kristin',
  'email': 'Kristin.Aasestad@ssb.no',
  'phone': '40902344'},
 'owningSection': '422 - Seksjon for næringslivets konjunkturer',
 'source': 'Reiselivsregioner 2020',
 'sourceId': 1439,
 'target': 'Kommuneinndeling 2020',
 'targetId': 1160,
 'changeTable': False,
 'lastModified': '2021-01-14T10:32:51.000+0000',
 'published': ['nb', 'nn', 'en'],
 'sourceLevel': None,
 'targetLevel': None,
 'description': 'Korrespondansetabellen viser sammenhengen mellom versjoner av to ulike kodeverk, f.eks. sammenhengen mellom Reiselivsregion 2020 og Kommuneinndeling 2020 (hvilke kommuner tilhører hvilke reiselivsregion). Dersom du ønsker å se forskjellen mellom to påfølgende versjoner av samme kodeliste, f.eks. mellom Kommuneinndeling 2020 og Kommuneinndeling 2018, finner du den under fanen «Endringer».',
 'changelogs': [{'changeOccured': '2020-12-17T13:28:28.000+0000',
   'description': 'Hamarøy l

Her der det mye informasjon. Selve korreponsetabellen ligger under correspondenceMaps. Vi lager en søkestreng til jmespath for å hente alle elementer under correspondenceMaps 

In [ ]:
search_string6 = 'correspondenceMaps[*]'

In [ ]:
res6 = jmespath.search(search_string6, dataset6)

Her bruker vi pd.DataFrame få å lage dataframe6

In [ ]:
dataframe6 = pd.DataFrame(res6)
dataframe6

## Eksempel 7. Hente variant som json

In [ ]:
URL7 = 'https://data.ssb.no/api/klass/v1/variants/1121.json'

In [ ]:
dataset7 = requests.get(url = URL7).json()

In [ ]:
# vise dataset 7
#dataset7

Det vi skal ha ligger under classificationItems

In [ ]:
search_string7 = 'classificationItems[*]'

In [ ]:
res7 = jmespath.search(search_string7, dataset7)

In [ ]:
dataframe7 = pd.DataFrame(res7)
dataframe7

In [ ]:
dataframe7 = dataframe7.dropna(axis='columns', how='all')
dataframe7

Spørsmål? Kontakt [statistikkbanken@ssb.no](statistikkbanken@ssb.no)